In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb  
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing

In [39]:
data = pd.read_csv('spotify_songs.csv')
df = pd.read_csv('Spotify Data/data-clean.csv')

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40560 entries, 0 to 40559
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   track_id           40560 non-null  object 
 1   time_signature     40560 non-null  float64
 2   chorus_hit         40560 non-null  float64
 3   sections           40560 non-null  float64
 4   target             40560 non-null  float64
 5   popularity         312 non-null    float64
 6   sm_target          40560 non-null  float64
 7   tiktok             40560 non-null  int64  
 8   spotify            40560 non-null  int64  
 9   track              40560 non-null  object 
 10  artist             40560 non-null  object 
 11  duration_ms        40560 non-null  float64
 12  danceability       40560 non-null  float64
 13  energy             40560 non-null  float64
 14  key                40560 non-null  object 
 15  loudness           40560 non-null  float64
 16  mode               405

In [40]:
data = data.drop(["popularity", "era", "main_parent_genre", "sections", "chorus_hit" ], axis=1)
data = data.drop(['tiktok', "sm_target", 'spotify', "track"], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40560 entries, 0 to 40559
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          40560 non-null  object 
 1   time_signature    40560 non-null  float64
 2   target            40560 non-null  float64
 3   artist            40560 non-null  object 
 4   duration_ms       40560 non-null  float64
 5   danceability      40560 non-null  float64
 6   energy            40560 non-null  float64
 7   key               40560 non-null  object 
 8   loudness          40560 non-null  float64
 9   mode              40560 non-null  object 
 10  speechiness       40560 non-null  float64
 11  acousticness      40560 non-null  float64
 12  instrumentalness  40560 non-null  float64
 13  liveness          40560 non-null  float64
 14  valence           40560 non-null  float64
 15  tempo             40560 non-null  float64
dtypes: float64(12), object(4)
memory usage: 

In [33]:
#data = pd.get_dummies(data)

In [42]:
data.head()

,track_id,time_signature,target,artist,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1dtKN6wwlolkM8XZy2y9C1,3.0,1.0,Garland Green,173533.0,0.417,0.620,D# / Eb,-7.727,major,0.0403,0.490,0.000000,0.0779,0.845,185.655
1,5hjsmSnUefdUqzsDogisiX,4.0,0.0,Serge Gainsbourg,213613.0,0.498,0.505,D# / Eb,-12.475,major,0.0337,0.018,0.107000,0.1760,0.797,101.801
2,6uk8tI6pwxxdVTNlNOJeJh,4.0,0.0,Lord Melody,223960.0,0.657,0.649,F,-13.392,major,0.0380,0.846,0.000004,0.1190,0.908,115.940
3,7aNjMJ05FvUXACPWZ7yJmv,4.0,0.0,Celia Cruz,157907.0,0.590,0.545,G,-12.058,minor,0.1040,0.706,0.024600,0.0610,0.967,105.592
4,1rQ0clvgkzWr001POOPJWx,4.0,0.0,P. Susheela,245600.0,0.515,0.765,B,-3.515,minor,0.1240,0.857,0.000872,0.2130,0.906,114.617


In [43]:
feature_cols=['acousticness', 'danceability', 'duration_ms', 'energy',
              'instrumentalness', 'liveness', 'loudness', 
              'speechiness', 'tempo', 'time_signature', 'valence',]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalized_df =scaler.fit_transform(data[feature_cols])

print(normalized_df[:2])

[[0.49196787 0.42206478 0.03811378 0.6199046  0.         0.0658215
  0.78355379 0.04197917 0.76900295 0.6        0.84839357]
 [0.01807229 0.50404858 0.04775985 0.50487572 0.107      0.1653144
  0.69396381 0.03510417 0.42167068 0.8        0.8002008 ]]


In [36]:
data.head()

,time_signature,target,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,...,key_C# / Db,key_D,key_D# / Eb,key_E,key_F,key_F# / Gb,key_G,key_G# / Ab,mode_major,mode_minor
0,3.0,1.0,173533.0,0.417,0.620,-7.727,0.0403,0.490,0.000000,0.0779,...,0,0,1,0,0,0,0,0,1,0
1,4.0,0.0,213613.0,0.498,0.505,-12.475,0.0337,0.018,0.107000,0.1760,...,0,0,1,0,0,0,0,0,1,0
2,4.0,0.0,223960.0,0.657,0.649,-13.392,0.0380,0.846,0.000004,0.1190,...,0,0,0,0,1,0,0,0,1,0
3,4.0,0.0,157907.0,0.590,0.545,-12.058,0.1040,0.706,0.024600,0.0610,...,0,0,0,0,0,0,1,0,0,1
4,4.0,0.0,245600.0,0.515,0.765,-3.515,0.1240,0.857,0.000872,0.2130,...,0,0,0,0,0,0,0,0,0,1


In [45]:
indices = pd.Series(data.index, index=data['track_id']).drop_duplicates()
# Create cosine similarity matrix based on given matrix
cosine = cosine_similarity(normalized_df)
def generate_recommendation(track_id, model_type=cosine):
    """
    Purpose: Function for song recommendations 
    Inputs: track_id and type of similarity model
    Output: Pandas series of recommended songs
    """
    # Get song indices
    index = indices[track_id]
    # Get list of songs for given track_id
    score = list(enumerate(model_type[index]))
    # Sort the most similar songs
    similarity_score = sorted(score, key=lambda x: x[1], reverse=True)
    # Select the top-10 recommended songs
    similarity_score = similarity_score[1:11]
    top_songs_index = [i[0] for i in similarity_score]
    # Top 10 recommended songs
    top_songs = data['track_id'].iloc[top_songs_index]
    return top_songs


In [46]:
print(indices.keys())

Index(['1dtKN6wwlolkM8XZy2y9C1', '5hjsmSnUefdUqzsDogisiX',
       '6uk8tI6pwxxdVTNlNOJeJh', '7aNjMJ05FvUXACPWZ7yJmv',
       '1rQ0clvgkzWr001POOPJWx', '32VBSoD2vcoIOiPEvAfFXU',
       '62eoocmGk59EMfuRcLBvUL', '0ecGwZA1ReAq4Mn3ZInaT4',
       '15gNqKkZN9Mq9AvnRqWJKb', '1hcMfYTsRTC4hIKbfosxjz',
       ...
       '4Ro98RCK90oHqqSZUnTFq5', '0aUWfpD3PlSv3FTTKcT2rN',
       '3bnVBN67NBEzedqQuWrpP4', '2QjOHCTQ1Jl3zawyYOpxh6',
       '4MofYf0f4ijlVV6elUW5S3', '4t1TljQWJ6ZuoSY67zVvBI',
       '2MShy1GSSgbmGUxADNIao5', '55qBw1900pZKfXJ6Q9A2Lc',
       '4o9npmYHrOF1rUxxTVH8h4', '2khIaVUkbMmDHB596lyMG3'],
      dtype='object', name='track_id', length=40560)


In [47]:
print("Recommended Songs:")
recommended_songs = generate_recommendation('5hjsmSnUefdUqzsDogisiX',cosine).values

Recommended Songs:


In [ ]:
# Create sigmoid kernel matrix based on given matrix
#sig_kernel = sigmoid_kernel(normalized_df)

#print("Recommended Songs:")
#print(generate_recommendation('Parallel Lines',sig_kernel).values)

In [48]:

# Filter the dataframe based on track_ids and target value
filtered_df = data[data['track_id'].isin(recommended_songs) & (df['target'] == 1)]

# Get the track_ids with target value 1
track_ids_with_target_1 = filtered_df['track_id'].tolist()

print(track_ids_with_target_1)

['2KSwmLHkgjZCE49YFkgspo', '41TqRBgF4Ahyr4vGmZVk8b', '55mQhobuwtY7lfLAXylg1k', '18bH4uo8CFY2yVwHbjal6l', '1A4eGVWeU3TUd1MxNdxU65', '6RueOnbP7XQyZaru8NBCdi', '5Abd8qlHxfliMmkggUCiSY']
